In [1]:
from ultralytics import YOLO
from sklearn.model_selection import train_test_split 

from data_img import BTV_Image_Dataset

from btv import *

from sklearn.ensemble import GradientBoostingClassifier

In [2]:
## Smalll data setup
ds = BTV_Image_Dataset("data/hardhat/test", max_dim=256, imvec_type='byol', embedding_weights='./runs/byol/caltech256+hh.pt',numpy=True,device=0)
idx_train, idx_test = train_test_split(
    np.arange(len(ds)), test_size=0.2
)
(X_train, y_train), (X_test, y_test) = ds[idx_train], ds[idx_test]

Detemining all classes...: 100%|██████████████████████████████| 1766/1766 [00:00<00:00, 2095.09it/s]


In [2]:
## Full setup
ds_train_byol = BTV_Image_Dataset("data/hardhat/train", max_dim=256, numpy=True, imvec_type='byol', embedding_weights='./runs/byol/caltech256+hh.pt')
ds_test_byol = BTV_Image_Dataset("data/hardhat/test", max_dim=256, numpy=True, imvec_type='byol', embedding_weights='./runs/byol/caltech256+hh.pt')
X_train, y_train, X_test, y_test = ds_train_byol.X, ds_train_byol.y, ds_test_byol.X, ds_test_byol.y

Detemining all classes...: 100%|██████████████████████████████| 1766/1766 [00:00<00:00, 2113.58it/s]


In [6]:
### pruning test
clf_knn = fit_dknn_toXy(X_train, y_train)
print(clf_knn.score(X_test,y_test))
X_train_selected, y_train_selected, selected_training_mask = prune_training_set(X_train, y_train, thresh=0.0001, discount_chance=False)
clf_knn.fit(X_train_selected, y_train_selected)
print(f"Pruned_set_size: {len(y_train_selected)}")
print(clf_knn.score(X_test,y_test))
### I think the latent space is WAY to fucking big. Used projection instead and ignored chance.

0.7853107344632768
Train set self-info: [      14.73      6.5322      5.1599]
Pruned_set_size: 74
0.7853107344632768


In [11]:
nn = NearestNeighbors(n_neighbors=1, metric="euclidean").fit(X_train)
_, train0_idxs = nn.kneighbors(X_test)
train0_idxs = train0_idxs.squeeze()
clf_knn.fit(X_train[train0_idxs],y_train[train0_idxs])
print(np.sum(prediction_info(y_test, clf_knn.predict_proba(X_test))))
print(clf_knn.score(X_test,y_test))
sorted_X, sorted_y, info = order_samples_by_info(X_train[train0_idxs],y_train[train0_idxs],clf_knn)
clf_knn.fit(X_train[train0_idxs][info>0],y_train[train0_idxs][info>0])
print(np.sum(prediction_info(y_test, clf_knn.predict_proba(X_test))))
print(clf_knn.score(X_test,y_test))

84.72278321267716
0.7627118644067796


ValueError: Found array with 0 sample(s) (shape=(0, 256)) while a minimum of 1 is required by KNeighborsClassifier.

In [92]:
## YOLO model testing
model = YOLO("yolo11n.pt")
model.train(data="data/hardhat/data.yaml")
model.test()

New https://pypi.org/project/ultralytics/8.3.52 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.35 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 7836MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=data/hardhat/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, s

train: Scanning /home/rakshat/Coding/better-than-vibes/data/hardhat/train/labels.cache... 5269 image


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /home/rakshat/Coding/better-than-vibes/data/hardhat/test/labels.cache... 1766 images, 


Plotting labels to runs/detect/train7/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train7
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       1766       6808      0.625      0.639      0.652      0.461



100 epochs completed in 1.262 hours.
Optimizer stripped from runs/detect/train7/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train7/weights/best.pt, 5.5MB

Validating runs/detect/train7/weights/best.pt...
Ultralytics 8.3.35 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 7836MiB)
YOLO11n summary (fused): 238 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|█████


                   all       1766       6808      0.626      0.646      0.658      0.464
                  head        339       1803      0.905       0.96      0.969      0.685
                helmet       1604       4863       0.93      0.963      0.982      0.694
                person         45        142     0.0408     0.0141     0.0234     0.0132
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/detect/train7


AttributeError: 'YOLO' object has no attribute 'test'